In [1]:
import pandas as pd

In [2]:
gkm_svm = pd.read_csv("/mnt/lab_data3/anusri/histone_expts/all_qtl_analysis/gm12878_sequence_sets/test_set/deltasv,/41588_2015_BFng3331_MOESM26_ESM.csv", header=0,sep=',')
print(gkm_svm.shape)


(28309, 18)


In [3]:
gkm_svm.head()

,chrom_hg18,pos_hg18,SNPname1,chrom_hg19,pos_hg19,SNPname2,allele1,allele2,gkm_SVM,kmer_SVM,abs_gkm_SVM,abs_kmer_SVM,CADD,GWAVA_region,GWAVA_tss,GWAVA_unmatched,GERP,label
0,chr1,846446,chr1.846446,chr1,856583,rs141671872,A,G,2.653531,7.938784,2.653531,7.938784,-1.522533,0.48,0.53,0.75,-0.682,1
1,chr1,901458,chr1.901458,chr1,911595,rs7417106,G,A,2.821422,6.027648,2.821422,6.027648,1.739152,0.29,0.36,0.89,2.150,1
2,chr1,1176365,chr1.1176365,chr1,1186502,rs6603785,T,A,-7.167236,-10.870018,7.167236,10.870018,-0.091378,0.42,0.49,0.66,0.451,1
3,chr1,1217275,chr1.1217275,chr1,1227412,rs112571439,A,G,-3.957382,2.387491,3.957382,2.387491,1.127472,0.34,0.43,0.49,2.430,1
4,chr1,1580438,chr1.1580438,chr1,1590575,rs9661285,A,G,-4.785596,-6.827102,4.785596,6.827102,0.665464,0.38,0.35,0.86,1.640,1


In [4]:
sum(gkm_svm["label"]==1)

574

In [5]:
sum(gkm_svm["label"]==-1)

27735

In [6]:
es = pd.read_csv("/mnt/lab_data2/anusri/variant-scorer/src/output/dsqtls_lcl/GSE31388_dsQtlTable.txt.gz", sep="\t")
enformer_preds = pd.read_csv("/mnt/lab_data2/anusri/chrombpnet/results/variant_data/dsqtls/process/fetch_enformer/enformer_predictions.tsv", sep="\t", header=0)
gkm_svm['rsid'] = gkm_svm["chrom_hg19"].astype(str) + "_" + gkm_svm["pos_hg19"].astype(str) + "_" + gkm_svm["allele1"].astype(str) + "_" + gkm_svm["allele2"].astype(str)  
es["rsid"] = es["Chr"]+"."+es["SNP"].astype(str)
gkm_svm = gkm_svm.merge(es, left_on=["SNPname1"], right_on='rsid', how="left").drop_duplicates('SNPname1')


In [7]:
gkm_svm.shape

(28309, 28)

In [8]:
gkm_svm.head()

,chrom_hg18,pos_hg18,SNPname1,chrom_hg19,pos_hg19,SNPname2,allele1,allele2,gkm_SVM,kmer_SVM,...,rsid_x,Chr,Start,End,SNP,Estimate,Std. Error,t value,Pr(>|t|),rsid_y
0,chr1,846446,chr1.846446,chr1,856583,rs141671872,A,G,2.653531,7.938784,...,chr1_856583_A_G,chr1,846400.0,846500.0,846446.0,0.9301,0.17900,5.197,2.013000e-06,chr1.846446
1,chr1,901458,chr1.901458,chr1,911595,rs7417106,G,A,2.821422,6.027648,...,chr1_911595_G_A,chr1,901400.0,901500.0,901458.0,-0.6285,0.08721,-7.207,6.010000e-10,chr1.901458
2,chr1,1176365,chr1.1176365,chr1,1186502,rs6603785,T,A,-7.167236,-10.870018,...,chr1_1186502_T_A,chr1,1176300.0,1176400.0,1176365.0,-0.7363,0.08125,-9.062,2.607000e-13,chr1.1176365
3,chr1,1217275,chr1.1217275,chr1,1227412,rs112571439,A,G,-3.957382,2.387491,...,chr1_1227412_A_G,chr1,1217200.0,1217300.0,1217275.0,-0.9236,0.13360,-6.912,2.041000e-09,chr1.1217275
4,chr1,1580438,chr1.1580438,chr1,1590575,rs9661285,A,G,-4.785596,-6.827102,...,chr1_1590575_A_G,chr1,1580400.0,1580500.0,1580438.0,-1.0890,0.15100,-7.211,5.902000e-10,chr1.1580438


In [9]:
gkm_svm = gkm_svm[["chrom_hg19", "pos_hg19", "allele1", "allele2",  "SNPname2", "gkm_SVM", "Estimate", "label"]]

In [10]:
gkm_svm.head(1)

,chrom_hg19,pos_hg19,allele1,allele2,SNPname2,gkm_SVM,Estimate,label
0,chr1,856583,A,G,rs141671872,2.653531,0.9301,1


In [11]:
enformer_preds.head(1)

,chr,pos,ref,alt,rsid,DNASE_SAD,DNASE_SAR
0,chr1,842250,A,G,rs148658266,-0.000669,-0.000892


In [12]:
merged_enf_1 = enformer_preds.merge(gkm_svm, left_on=["rsid", "chr", "pos", "ref", "alt"], right_on=["SNPname2", "chrom_hg19", "pos_hg19", "allele2", "allele1"])
merged_enf_2 = enformer_preds.merge(gkm_svm, left_on=["rsid", "chr", "pos", "ref", "alt"], right_on=["SNPname2", "chrom_hg19", "pos_hg19", "allele1", "allele2"])
merged_enf_2["DNASE_SAD"] = merged_enf_2["DNASE_SAD"]*-1
merged_enf_2["DNASE_SAR"] = merged_enf_2["DNASE_SAR"]*-1
merged_enf = pd.concat([merged_enf_1, merged_enf_2]).drop_duplicates()

In [13]:
merged_enf.head(1)

,chr,pos,ref,alt,rsid,DNASE_SAD,DNASE_SAR,chrom_hg19,pos_hg19,allele1,allele2,SNPname2,gkm_SVM,Estimate,label
0,chr1,842250,A,G,rs148658266,-0.000669,-0.000892,chr1,842250,G,A,rs148658266,-3.62651,NaN,-1


In [14]:
rsid_samples = merged_enf["rsid"].unique()
gkm_svm["IsUsed"] = False
gkm_svm.loc[gkm_svm["SNPname2"].isin(rsid_samples),"IsUsed"] = True

print(sum(gkm_svm["IsUsed"]))
print(sum(gkm_svm["label"]==1 & gkm_svm["IsUsed"]))
print(sum((gkm_svm["label"]==-1) & gkm_svm["IsUsed"]))


27373
560
26813


In [15]:
data = gkm_svm.merge(merged_enf[["chrom_hg19", "pos_hg19", "allele1", "allele2", "DNASE_SAD", "DNASE_SAR"]] , on=["chrom_hg19", "pos_hg19", "allele1", "allele2"], how="left")
print(data.shape)


(28309, 11)


In [16]:
import numpy as np
sum(~data["IsUsed"] & np.isnan(data["DNASE_SAD"]))

936

In [17]:
data.head()

,chrom_hg19,pos_hg19,allele1,allele2,SNPname2,gkm_SVM,Estimate,label,IsUsed,DNASE_SAD,DNASE_SAR
0,chr1,856583,A,G,rs141671872,2.653531,0.9301,1,True,0.000342,0.000450
1,chr1,911595,G,A,rs7417106,2.821422,-0.6285,1,True,-0.001241,-0.001526
2,chr1,1186502,T,A,rs6603785,-7.167236,-0.7363,1,True,-0.006530,-0.007840
3,chr1,1227412,A,G,rs112571439,-3.957382,-0.9236,1,True,-0.001878,-0.002290
4,chr1,1590575,A,G,rs9661285,-4.785596,-1.0890,1,True,-0.008736,-0.010230


In [18]:
atac_preds="/mnt/lab_data2/anusri/variant-scorer/src/output/dsqtls_lcl/ATAC/summary.mean.variant_scores_new_2.tsv"
dnase_preds="/mnt/lab_data2/anusri/variant-scorer/src/output/dsqtls_lcl/DNASE_NEW/summary.mean.variant_scores_new_3.tsv"
atac_preds_50m="/mnt/lab_data2/anusri/variant-scorer/src/output/dsqtls_lcl/ATAC_50M/summary.mean.variant_scores_new_2.tsv"

atac_preds_250m="/mnt/lab_data2/anusri/variant-scorer/src/output/dsqtls_lcl/ATAC_250M/summary.mean.variant_scores_new_2.tsv"
atac_preds_100m="/mnt/lab_data2/anusri/variant-scorer/src/output/dsqtls_lcl/ATAC_100M/summary.mean.variant_scores_new_2.tsv"
atac_preds_25m="/mnt/lab_data2/anusri/variant-scorer/src/output/dsqtls_lcl/ATAC_25M/summary.mean.variant_scores_new_2.tsv"
atac_preds_5m="/mnt/lab_data2/anusri/variant-scorer/src/output/dsqtls_lcl/ATAC_5M/summary.mean.variant_scores_new_2.tsv"


In [19]:
def get_sel_columns_and_append(df_f, out_df, title):
    
    keys = ["abs_logfc_x_jsd_x_max_percentile.mean",
        "abs_logfc_x_jsd_x_max_percentile.mean.pval",
        "abs_logfc_x_jsd.mean" , 
        "jsd_x_max_percentile.mean", 
        "abs_logfc_x_max_percentile.mean", 
        "jsd.mean", 
        "abs_logfc.mean",
        "max_percentile.mean"]
    
    df = pd.read_csv(df_f, sep='\t', header=0)

    for key in keys:
        
        out_df.loc[:,title+"."+key] = df[key]
    
    
    out_df[title+"."+"logfc.mean"] = df["logfc.mean"]*-1
    out_df[title+"."+"sign.jsd.mean"] = np.sign(df["logfc.mean"])*df["jsd.mean"]*-1
    
    return out_df
    

In [20]:
data = data[['chrom_hg19', 'pos_hg19', 'allele1', 'allele2', 'SNPname2', 'Estimate', 'label',  'IsUsed', 'gkm_SVM', 'DNASE_SAD', 'DNASE_SAR']]


In [21]:
data = get_sel_columns_and_append(atac_preds, data, "ENCSR637XSC")
data = get_sel_columns_and_append(dnase_preds, data, "ENCSR000EMT")
data = get_sel_columns_and_append(atac_preds_50m, data, "ENCSR637XSC.sub.50M")

In [22]:
def get_sel_columns_and_append(df_f, out_df, title):
    
    keys = ["abs_logfc_x_jsd_x_max_percentile.mean",
        "abs_logfc_x_jsd_x_max_percentile.mean.pval"]
    
    df = pd.read_csv(df_f, sep='\t', header=0)

    for key in keys:
        
        out_df.loc[:,title+"."+key] = df[key]
    
    
    out_df[title+"."+"logfc.mean"] = df["logfc.mean"]*-1
    out_df[title+"."+"sign.jsd.mean"] = np.sign(df["logfc.mean"])*df["jsd.mean"]*-1
        
    return out_df


In [23]:
data = get_sel_columns_and_append(atac_preds_250m, data, "ENCSR637XSC.sub.250M")
data = get_sel_columns_and_append(atac_preds_100m, data, "ENCSR637XSC.sub.100M")
data = get_sel_columns_and_append(atac_preds_25m, data, "ENCSR637XSC.sub.25M")
data = get_sel_columns_and_append(atac_preds_5m, data, "ENCSR637XSC.sub.5M")


In [24]:
data.shape

(28309, 57)

In [25]:
data.head(1)

,chrom_hg19,pos_hg19,allele1,allele2,SNPname2,Estimate,label,IsUsed,gkm_SVM,DNASE_SAD,...,ENCSR637XSC.sub.100M.logfc.mean,ENCSR637XSC.sub.100M.sign.jsd.mean,ENCSR637XSC.sub.25M.abs_logfc_x_jsd_x_max_percentile.mean,ENCSR637XSC.sub.25M.abs_logfc_x_jsd_x_max_percentile.mean.pval,ENCSR637XSC.sub.25M.logfc.mean,ENCSR637XSC.sub.25M.sign.jsd.mean,ENCSR637XSC.sub.5M.abs_logfc_x_jsd_x_max_percentile.mean,ENCSR637XSC.sub.5M.abs_logfc_x_jsd_x_max_percentile.mean.pval,ENCSR637XSC.sub.5M.logfc.mean,ENCSR637XSC.sub.5M.sign.jsd.mean
0,chr1,856583,A,G,rs141671872,0.9301,1,True,2.653531,0.000342,...,0.61987,0.091883,0.00484,0.018692,0.291935,0.067401,0.00136,0.065265,0.065098,0.049902


In [26]:
data.columns

Index(['chrom_hg19', 'pos_hg19', 'allele1', 'allele2', 'SNPname2', 'Estimate',
       'label', 'IsUsed', 'gkm_SVM', 'DNASE_SAD', 'DNASE_SAR',
       'ENCSR637XSC.abs_logfc_x_jsd_x_max_percentile.mean',
       'ENCSR637XSC.abs_logfc_x_jsd_x_max_percentile.mean.pval',
       'ENCSR637XSC.abs_logfc_x_jsd.mean',
       'ENCSR637XSC.jsd_x_max_percentile.mean',
       'ENCSR637XSC.abs_logfc_x_max_percentile.mean', 'ENCSR637XSC.jsd.mean',
       'ENCSR637XSC.abs_logfc.mean', 'ENCSR637XSC.max_percentile.mean',
       'ENCSR637XSC.logfc.mean', 'ENCSR637XSC.sign.jsd.mean',
       'ENCSR000EMT.abs_logfc_x_jsd_x_max_percentile.mean',
       'ENCSR000EMT.abs_logfc_x_jsd_x_max_percentile.mean.pval',
       'ENCSR000EMT.abs_logfc_x_jsd.mean',
       'ENCSR000EMT.jsd_x_max_percentile.mean',
       'ENCSR000EMT.abs_logfc_x_max_percentile.mean', 'ENCSR000EMT.jsd.mean',
       'ENCSR000EMT.abs_logfc.mean', 'ENCSR000EMT.max_percentile.mean',
       'ENCSR000EMT.logfc.mean', 'ENCSR000EMT.sign.jsd.mean',
  

In [27]:
import pickle as pkl
import os
enformer_recompute = pkl.load(open(os.path.join("/mnt/lab_data2/anusri/variant-scorer/src/output/dsqtls_lcl/enformer_preds_small_window/predictions_at_snp.pkl"),'rb'))
mean_change=[]
for i in range(data.shape[0]):
    mean_change.append(np.log2( (enformer_recompute["ref_logcount_preds"][i,440+4:456-4].sum()+1) / (enformer_recompute["alt_logcount_preds"][i,440+4:456-4].sum()+1)))
    
mean_change = np.array(mean_change)
                       
      
data["pred.enformer.encsr000emt.varscore.local_logfc"] = mean_change

In [28]:
dict_rename = {"chrom_hg19": "var.chrom",
                            "SNPname2": "var.rsid",
                            "DNASE_SAD": "pred.enformer.encsr000emt.varscore.global_sad",
                            "DNASE_SAR": "pred.enformer.encsr000emt.varscore.global_sar",
                            "pos_hg19": "var.pos_hg19",
                            "allele1": "var.allele1",
                            "allele2": "var.allele2",
                            "Estimate": "obs.estimate",
                            "label" : "obs.label",
                            "IsUsed": "var.isused",
                            "gkm_SVM": "pred.gkmsvm.encsr000emt.varscore.deltasvm",
                            "ENCSR637XSC.abs_logfc_x_jsd_x_max_percentile.mean": "pred.chrombpnet.encsr637xsc.varscore.ips",
                            "ENCSR000EMT.abs_logfc_x_jsd_x_max_percentile.mean": "pred.chrombpnet.encsr000emt.varscore.ips",
                            "ENCSR637XSC.sub.50M.abs_logfc_x_jsd_x_max_percentile.mean": "pred.chrombpnet.encsr637xsc_50m.varscore.ips",

                            "ENCSR637XSC.abs_logfc_x_jsd_x_max_percentile.mean.pval": "pred.chrombpnet.encsr637xsc.varscore.pval",
                            "ENCSR000EMT.abs_logfc_x_jsd_x_max_percentile.mean.pval": "pred.chrombpnet.encsr000emt.varscore.pval",
                            "ENCSR637XSC.sub.50M.abs_logfc_x_jsd_x_max_percentile.mean.pval": "pred.chrombpnet.encsr637xsc_50m.varscore.pval",

                            "ENCSR637XSC.logfc.mean": "pred.chrombpnet.encsr637xsc.varscore.logfc",
                            "ENCSR000EMT.logfc.mean": "pred.chrombpnet.encsr000emt.varscore.logfc",
                            "ENCSR637XSC.sub.50M.logfc.mean": "pred.chrombpnet.encsr637xsc_50m.varscore.logfc",

                            "ENCSR637XSC.sign.jsd.mean": "pred.chrombpnet.encsr637xsc.varscore.jsd",
                            "ENCSR000EMT.sign.jsd.mean": "pred.chrombpnet.encsr000emt.varscore.jsd",
                            "ENCSR637XSC.sub.50M.sign.jsd.mean": "pred.chrombpnet.encsr637xsc_50m.varscore.jsd",

                            "ENCSR637XSC.max_percentile.mean": "pred.chrombpnet.encsr637xsc.varscore.aaq",
                            "ENCSR000EMT.max_percentile.mean": "pred.chrombpnet.encsr000emt.varscore.aaq",
                            "ENCSR637XSC.sub.50M.max_percentile.mean": "pred.chrombpnet.encsr637xsc_50m.varscore.aaq",
               
                            "ENCSR637XSC.abs_logfc_x_jsd.mean": "pred.chrombpnet.encsr637xsc.varscore.ies",
                            "ENCSR000EMT.abs_logfc_x_jsd.mean": "pred.chrombpnet.encsr000emt.varscore.ies",
                            "ENCSR637XSC.sub.50M.abs_logfc_x_jsd.mean": "pred.chrombpnet.encsr637xsc_50m.varscore.ies",

                            "ENCSR637XSC.jsd_x_max_percentile.mean": "pred.chrombpnet.encsr637xsc.varscore.jsd_x_aaq",
                            "ENCSR000EMT.jsd_x_max_percentile.mean": "pred.chrombpnet.encsr000emt.varscore.jsd_x_aaq",
                            "ENCSR637XSC.sub.50M.jsd_x_max_percentile.mean": "pred.chrombpnet.encsr637xsc_50m.varscore.jsd_x_aaq",
               
                            "ENCSR637XSC.abs_logfc_x_max_percentile.mean": "pred.chrombpnet.encsr637xsc.varscore.logfc_x_aaq",
                            "ENCSR000EMT.abs_logfc_x_max_percentile.mean": "pred.chrombpnet.encsr000emt.varscore.logfc_x_aaq",
                            "ENCSR637XSC.sub.50M.abs_logfc_x_max_percentile.mean": "pred.chrombpnet.encsr637xsc_50m.varscore.logfc_x_aaq"}


keyds = list(dict_rename.keys())
for key in keyds:
    if "50M" in key:
        for sub in ["250M", "100M", "25M", "5M"]:
            #print(dict_rename[key].replace("50M".lower(), sub.lower()))
            dict_rename[key.replace("50M", sub)] = dict_rename[key].replace("50M".lower(), sub.lower())
    


In [29]:
#dict_rename

In [30]:
data = data.rename(columns=dict_rename)

In [31]:
data.columns

Index(['var.chrom', 'var.pos_hg19', 'var.allele1', 'var.allele2', 'var.rsid',
       'obs.estimate', 'obs.label', 'var.isused',
       'pred.gkmsvm.encsr000emt.varscore.deltasvm',
       'pred.enformer.encsr000emt.varscore.global_sad',
       'pred.enformer.encsr000emt.varscore.global_sar',
       'pred.chrombpnet.encsr637xsc.varscore.ips',
       'pred.chrombpnet.encsr637xsc.varscore.pval',
       'pred.chrombpnet.encsr637xsc.varscore.ies',
       'pred.chrombpnet.encsr637xsc.varscore.jsd_x_aaq',
       'pred.chrombpnet.encsr637xsc.varscore.logfc_x_aaq',
       'ENCSR637XSC.jsd.mean', 'ENCSR637XSC.abs_logfc.mean',
       'pred.chrombpnet.encsr637xsc.varscore.aaq',
       'pred.chrombpnet.encsr637xsc.varscore.logfc',
       'pred.chrombpnet.encsr637xsc.varscore.jsd',
       'pred.chrombpnet.encsr000emt.varscore.ips',
       'pred.chrombpnet.encsr000emt.varscore.pval',
       'pred.chrombpnet.encsr000emt.varscore.ies',
       'pred.chrombpnet.encsr000emt.varscore.jsd_x_aaq',
       'pre

In [32]:
use_cols = ['var.chr', 'var.pos_hg19', 'var.allele1', 'var.allele2', 'var.rsid', 'obs.label', 'var.isused',
       'obs.estimate', 
       'pred.gkmsvm.encsr000emt.varscore.deltasvm',
       'pred.enformer.encsr000emt.varscore.global_sad',
       'pred.enformer.encsr000emt.varscore.global_sar',
       'pred.enformer.encsr000emt.varscore.local_logfc',
       'pred.chrombpnet.encsr000emt.varscore.logfc',
       'pred.chrombpnet.encsr000emt.varscore.jsd', 
       'pred.chrombpnet.encsr000emt.varscore.ips',
       'pred.chrombpnet.encsr000emt.varscore.aaq',
       'pred.chrombpnet.encsr000emt.varscore.ies',
       'pred.chrombpnet.encsr000emt.varscore.jsd_x_aaq',
       'pred.chrombpnet.encsr000emt.varscore.logfc_x_aaq', 

       'pred.chrombpnet.encsr637xsc.varscore.logfc',
       'pred.chrombpnet.encsr637xsc.varscore.jsd', 
       'pred.chrombpnet.encsr637xsc.varscore.ips',
       'pred.chrombpnet.encsr637xsc.varscore.aaq',
       'pred.chrombpnet.encsr637xsc.varscore.ies',
       'pred.chrombpnet.encsr637xsc.varscore.jsd_x_aaq',
       'pred.chrombpnet.encsr637xsc.varscore.logfc_x_aaq', 

       'pred.chrombpnet.encsr637xsc_250m.varscore.logfc',
       'pred.chrombpnet.encsr637xsc_250m.varscore.jsd', 
       'pred.chrombpnet.encsr637xsc_250m.varscore.ips',


       'pred.chrombpnet.encsr637xsc_100m.varscore.logfc',
       'pred.chrombpnet.encsr637xsc_100m.varscore.jsd', 
       'pred.chrombpnet.encsr637xsc_100m.varscore.ips',

       'pred.chrombpnet.encsr637xsc_50m.varscore.logfc',
       'pred.chrombpnet.encsr637xsc_50m.varscore.jsd', 
       'pred.chrombpnet.encsr637xsc_50m.varscore.ips',
       'pred.chrombpnet.encsr637xsc_50m.varscore.aaq',
       'pred.chrombpnet.encsr637xsc_50m.varscore.ies',
       'pred.chrombpnet.encsr637xsc_50m.varscore.jsd_x_aaq',
       'pred.chrombpnet.encsr637xsc_50m.varscore.logfc_x_aaq', 

       'pred.chrombpnet.encsr637xsc_25m.varscore.logfc',
       'pred.chrombpnet.encsr637xsc_25m.varscore.jsd', 
       'pred.chrombpnet.encsr637xsc_25m.varscore.ips',

       'pred.chrombpnet.encsr637xsc_5m.varscore.logfc',
       'pred.chrombpnet.encsr637xsc_5m.varscore.jsd', 
       'pred.chrombpnet.encsr637xsc_5m.varscore.ips'  ]


 


In [33]:
dataf = data[use_cols]

#data.columns

In [34]:
dataf.head()

,var.chrom,var.pos_hg19,var.allele1,var.allele2,var.rsid,obs.label,var.isused,obs.estimate,pred.gkmsvm.encsr000emt.varscore.deltasvm,pred.enformer.encsr000emt.varscore.global_sad,...,pred.chrombpnet.encsr637xsc_50m.varscore.aaq,pred.chrombpnet.encsr637xsc_50m.varscore.ies,pred.chrombpnet.encsr637xsc_50m.varscore.jsd_x_aaq,pred.chrombpnet.encsr637xsc_50m.varscore.logfc_x_aaq,pred.chrombpnet.encsr637xsc_25m.varscore.logfc,pred.chrombpnet.encsr637xsc_25m.varscore.jsd,pred.chrombpnet.encsr637xsc_25m.varscore.ips,pred.chrombpnet.encsr637xsc_5m.varscore.logfc,pred.chrombpnet.encsr637xsc_5m.varscore.jsd,pred.chrombpnet.encsr637xsc_5m.varscore.ips
0,chr1,856583,A,G,rs141671872,1,True,0.9301,2.653531,0.000342,...,0.248566,0.044994,0.021877,0.127298,0.291935,0.067401,0.004840,0.065098,0.049902,0.001360
1,chr1,911595,G,A,rs7417106,1,True,-0.6285,2.821422,-0.001241,...,0.726685,0.002535,0.027762,0.056287,0.093192,0.036053,0.002239,0.045314,0.034489,0.000898
2,chr1,1186502,T,A,rs6603785,1,True,-0.7363,-7.167236,-0.006530,...,0.696101,0.021468,0.029658,0.349964,-0.402188,-0.038791,0.008839,-0.163023,-0.039080,0.002210
3,chr1,1227412,A,G,rs112571439,1,True,-0.9236,-3.957382,-0.001878,...,0.521835,0.265543,0.137196,0.518532,-0.622704,-0.222396,0.086945,-0.259064,-0.172960,0.027583
4,chr1,1590575,A,G,rs9661285,1,True,-1.0890,-4.785596,-0.008736,...,0.988211,0.006033,0.053680,0.103350,-0.061906,-0.050589,0.003275,-0.032300,-0.023135,0.000802


In [35]:
dataf_final = dataf[dataf["var.isused"]]

In [36]:
dataf_final.shape

(27373, 45)

In [37]:
dataf_final = dataf_final.drop(["var.isused"], axis=1)

In [38]:
dataf_final.head()

,var.chrom,var.pos_hg19,var.allele1,var.allele2,var.rsid,obs.label,obs.estimate,pred.gkmsvm.encsr000emt.varscore.deltasvm,pred.enformer.encsr000emt.varscore.global_sad,pred.enformer.encsr000emt.varscore.global_sar,...,pred.chrombpnet.encsr637xsc_50m.varscore.aaq,pred.chrombpnet.encsr637xsc_50m.varscore.ies,pred.chrombpnet.encsr637xsc_50m.varscore.jsd_x_aaq,pred.chrombpnet.encsr637xsc_50m.varscore.logfc_x_aaq,pred.chrombpnet.encsr637xsc_25m.varscore.logfc,pred.chrombpnet.encsr637xsc_25m.varscore.jsd,pred.chrombpnet.encsr637xsc_25m.varscore.ips,pred.chrombpnet.encsr637xsc_5m.varscore.logfc,pred.chrombpnet.encsr637xsc_5m.varscore.jsd,pred.chrombpnet.encsr637xsc_5m.varscore.ips
0,chr1,856583,A,G,rs141671872,1,0.9301,2.653531,0.000342,0.000450,...,0.248566,0.044994,0.021877,0.127298,0.291935,0.067401,0.004840,0.065098,0.049902,0.001360
1,chr1,911595,G,A,rs7417106,1,-0.6285,2.821422,-0.001241,-0.001526,...,0.726685,0.002535,0.027762,0.056287,0.093192,0.036053,0.002239,0.045314,0.034489,0.000898
2,chr1,1186502,T,A,rs6603785,1,-0.7363,-7.167236,-0.006530,-0.007840,...,0.696101,0.021468,0.029658,0.349964,-0.402188,-0.038791,0.008839,-0.163023,-0.039080,0.002210
3,chr1,1227412,A,G,rs112571439,1,-0.9236,-3.957382,-0.001878,-0.002290,...,0.521835,0.265543,0.137196,0.518532,-0.622704,-0.222396,0.086945,-0.259064,-0.172960,0.027583
4,chr1,1590575,A,G,rs9661285,1,-1.0890,-4.785596,-0.008736,-0.010230,...,0.988211,0.006033,0.053680,0.103350,-0.061906,-0.050589,0.003275,-0.032300,-0.023135,0.000802


In [39]:
assert (sum(dataf.columns.isin(use_cols))==dataf.shape[1])



In [40]:
dataf.to_csv("full_variants/dsqtls.yoruba.lcls.benchmarking.all.tsv", sep='\t', header=True, index=False)

In [41]:
dataf_final.to_csv("filtered_variants/dsqtls.yoruba.lcls.benchmarking.chrombpnet.tsv", sep='\t', header=True, index=False)

In [49]:

readme_data = {'var.chrom' : "Chromosome of the variant (GRCh37)", 
        'var.pos_hg19': "Position of the variant (GRCh37, 1-based)",
        'var.allele1': "Allele 1 for the variant", 
        'var.allele2': "Allele 2 for the variant",        
        'var.rsid': "dbSNP rsid identifier", 
       'obs.label': "True if variant has significant observed effect size", 
       'var.isused': "True if variant is used in final ChromBPNet benchmarking",
       'obs.estimate': "Observed effect size", 
       'pred.gkmsvm.encsr000emt.varscore.deltasvm' : "deltasvm predictions in encsr000emt",
       'pred.enformer.encsr000emt.varscore.global_sad': "Enformer SAD predictions (downloaded) in encsr000emt",
       'pred.enformer.encsr000emt.varscore.global_sar': "Enformer SAR predictions (downloaded) in encsr000emt",
       'pred.enformer.encsr000emt.varscore.local_logfc': "Enformer logFC (local) predictions (re-computed) in encsr000emt",
       'pred.chrombpnet.xxx.varscore.logfc': "ChromBPNet logFC predictions in encid xxx",
       'pred.chrombpnet.xxx.varscore.jsd':  "ChromBPNet JSD predictions in encid xxx", 
       'pred.chrombpnet.xxx.varscore.ips': "ChromBPNet IPS predictions in encid xxx",
       'pred.chrombpnet.xxx.varscore.aaq': "ChromBPNet AAQ predictions in encid xxx",
       'pred.chrombpnet.xxx.varscore.ies': "ChromBPNet IES predictions in encid xxx",
       'pred.chrombpnet.xxx.varscore.jsd_x_aaq': "ChromBPNet JSD x AAQ predictions in encid xxx",
       'pred.chrombpnet.xxx.varscore.logfc_x_aaq': "ChromBPNet logFC x AAQ predictions in encid xxx"}

# encsr637xsc_5m
# encsr637xsc_25m
# encsr637xsc_50m
# encsr637xsc_100m
# encsr637xsc_250m
# encsr637xsc

In [54]:
f = open("full_variants/dsqtls.yoruba.lcls.benchmarking.all.README", "w")
f.write("# TSV Field Header Descriptions \n \n")
for val in readme_data:
    print(val)
    f.write(val+" : "+readme_data[val]+"\n")
f.write("\n")
f.write("# Experiment-Specific Headers and IDs  \n")
f.close()

var.chrom
var.pos_hg19
var.allele1
var.allele2
var.rsid
obs.label
var.isused
obs.estimate
pred.gkmsvm.encsr000emt.varscore.deltasvm
pred.enformer.encsr000emt.varscore.global_sad
pred.enformer.encsr000emt.varscore.global_sar
pred.enformer.encsr000emt.varscore.local_logfc
pred.chrombpnet.xxx.varscore.logfc
pred.chrombpnet.xxx.varscore.jsd
pred.chrombpnet.xxx.varscore.ips
pred.chrombpnet.xxx.varscore.aaq
pred.chrombpnet.xxx.varscore.ies
pred.chrombpnet.xxx.varscore.jsd_x_aaq
pred.chrombpnet.xxx.varscore.logfc_x_aaq
